## Baseline univariate time-series model  

### example: persistence forecast for monthly car sales dataset

In [6]:
from math import sqrt
from numpy import median, mean, std
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot

### Methods for splitting the dataset into train and test + evaluation metric calculation + model fit&predict

In [48]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

# difference dataset
def difference(data, interval):
    return [data[i] - data[i - interval] for i in range(interval, len(data))]

# fit a model
def model_fit(train, config):
    return None

# forecast with a pre-fit model
# naive model: median of values: sales at 1, 2 and 3 years ago
def model_predict(model, history, config):
    values = list()
    for offset in config:
        values.append(history[-offset])
    return median(values)


### Methods for model evaluation 

In [177]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model
    model = model_fit(train, cfg)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = model_predict(model, history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_rmse(test, predictions)
    print(' > %.3f' % error)
    
    return error, predictions

# repeat evaluation of a config
def repeat_evaluate(data, config, n_test, n_repeats=30):
    # fit and evaluate the model n times
    predictions_matrix = []
    scores = []
    #scores = [walk_forward_validation(data, n_test, config) for _ in range(n_repeats)]
    for _ in range(n_repeats):
        score, predictions = walk_forward_validation(data, n_test, config) 
        scores.append(score)
        predictions_matrix.append(predictions)
    
    return scores, predictions_matrix

# summarize model performance
def summarize_scores(name, scores):
    # print a summary
    scores_m, score_std = mean(scores), std(scores)
    print('%s: %.3f RMSE (+/- %.3f)' % (name, scores_m, score_std))
    # box and whisker plot
    pyplot.boxplot(scores)
    pyplot.show()


In [194]:
def make_line_plots(title, dataset, attributes_to_plot=[], x_axis_label='x', y_axis_label='y', 
                        background_color=None, legend_location="top_left", line_colors=['red']):
        try:
            from bokeh.plotting import figure, show, output_file
            from bokeh.models import HoverTool
            from bokeh.io import output_notebook

            output_notebook()

            p = figure(title=title, tools='', background_fill_color=background_color)
            p.legend.location = legend_location

            x_values = dataset.index
            i = 0
            for attribute in attributes_to_plot:
                    attribute_values = dataset[attribute]
                    p.line(x_values, attribute_values, legend=attribute, line_dash=[4, 4], line_color=line_colors[i], 
                            line_width=2)
                    i += 1


            p.y_range.start = 0
            #p.legend.location = "center_right"
            p.legend.background_fill_color = background_color
            p.xaxis.axis_label = x_axis_label
            p.yaxis.axis_label = y_axis_label
            p.grid.grid_line_color="white"

            p.add_tools(HoverTool())
            p.select_one(HoverTool).tooltips = [
                                    (x_axis_label, '@x'),
                                    (y_axis_label, '@y'),
                                ]
            p.legend.click_policy="hide"

            show(p)

        except Exception as exc:
            logger.exception('raised exception at {}: {}'.format(logger.name+'.'+make_line_plots.__name__, exc))


### Example with a CHINA EV sales dataset 

In [195]:
#data source: https://datasetsearch.research.google.com/search?query=univariate%20time%20series&docid=Z2B66b7T3lUIl0y6AAAAAA%3D%3D&filters=bm9uZQ%3D%3D&property=aXNfYWNjZXNzaWJsZV9mb3JfZnJlZQ%3D%3D
ev_sales_data = read_csv(r'.\datasets\china_electric_vehicles_sales.csv')

ev_sales_data.iloc[:6]

,Year/Month,BEV,sales
0,2011/01,588,310
1,2011/02,47,417
2,2011/03,159,286
3,2011/04,173,162
4,2011/05,305,470
5,2011/06,319,273


In [196]:
ev_sales_data['Date'] = pd.to_datetime(ev_sales_data['Year/Month'])
ev_sales_data.set_index('Date', inplace=True)
ev_sales_data.drop(columns=['Year/Month'], inplace=True)

sales_series_values = ev_sales_data['sales'].values
ev_sales_data.head(5)

,BEV,sales
Date,,
2011-01-01,588,310
2011-02-01,47,417
2011-03-01,159,286
2011-04-01,173,162
2011-05-01,305,470


In [197]:
ev_sales_data['year'] = pd.Series(ev_sales_data.index).apply(lambda x: x.year).values
ev_sales_data['month'] = pd.Series(ev_sales_data.index).apply(lambda x: x.month).values
ev_sales_data.head(5)

,BEV,sales,year,month
Date,,,,
2011-01-01,588,310,2011,1
2011-02-01,47,417,2011,2
2011-03-01,159,286,2011,3
2011-04-01,173,162,2011,4
2011-05-01,305,470,2011,5


In [198]:
make_line_plots('EV monthly sales', ev_sales_data, attributes_to_plot=['sales'], x_axis_label='Month', y_axis_label='Sales')

Loading BokehJS ...

In [200]:
# data split
n_test = 12
# define config
config = [12, 24, 36]
# grid search
scores, predictions_matrix = repeat_evaluate(sales_series_values, config, n_test)


> 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991
 > 8142.991


In [ ]:
# summarize scores
summarize_scores('persistence', scores)

#### Vemos que lo que ocurre es que nuestro modelo es determinista, así que aunque utilicemos varias veces el walk-forward validation method con el consiguiente entrenamiento del modelo, obtenemos las mismas predicciones y los mismos errores medios cuadráticos <p>
Formamos el dataset con los valores a predecir VS predichos

In [201]:
ev_sales_forecast_df = pd.DataFrame()

test_rmse = measure_rmse(ev_sales_data['sales'][-n_test:].values, predictions_matrix[-1])
if int(test_rmse) == int(scores[-1]): 
    ev_sales_forecast_df['sales_test_values'] = ev_sales_data['sales'][-n_test:]
    ev_sales_forecast_df['sales_baseline_predictions'] = predictions_matrix[-1]

ev_sales_forecast_df

,sales_test_values,sales_baseline_predictions
Date,,
2015-01-01,3148,436.0
2015-02-01,6334,387.0
2015-03-01,4079,826.0
2015-04-01,5730,635.0
2015-05-01,6354,1517.0
2015-06-01,8308,1037.0
2015-07-01,7338,1709.0
2015-08-01,12085,1701.0
2015-09-01,12736,1306.0


### plot real values VS predicted ones

In [202]:
make_line_plots('EV monthly sales predictions', ev_sales_forecast_df, attributes_to_plot=['sales_test_values', 'sales_baseline_predictions'], x_axis_label='Month',        y_axis_label='Sales', line_colors=['blue', 'red'])

Loading BokehJS ...

C:\Users\gcabreram\AppData\Local\Continuum\anaconda3\envs\deep_learning_tf2_env\lib\site-packages\bokeh\models\plots.py:767: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
